In [11]:
from dotenv import load_dotenv
load_dotenv()

import re
import os, json

from textwrap import dedent
from pprint import pprint

import warnings
warnings.filterwarnings("ignore")

In [12]:
from typing import Dict, Any, List
from langchain_core.messages import HumanMessage, AIMessage, SystemMessage
from langchain_core.tools import tool
from langchain_core.prompts import ChatPromptTemplate
from langchain_ollama import ChatOllama, OllamaEmbeddings
from langchain_community.vectorstores import FAISS
from langgraph.graph import StateGraph, MessagesState, START, END
from langgraph.prebuilt import ToolNode, tools_condition
from langgraph.checkpoint.memory import MemorySaver
from langchain_openai import ChatOpenAI

embeddings_model = OllamaEmbeddings(model="bge-m3:latest") 

cafe_db = FAISS.load_local(
    "./db/cafe_db", 
    embeddings_model, 
    allow_dangerous_deserialization=True
)

In [13]:
@tool
def search_cafe_menu(query: str) -> str:
    """카페 메뉴를 검색하는 도구입니다. 검색할 메뉴 이름이나 키워드를 입력하세요 (예: 아메리카노, 라떼, 디저트)"""
    try:
        # Vector DB에서 유사한 메뉴 검색 (상위 4개 결과)
        docs = cafe_db.similarity_search(query, k=4)
        
        if len(docs) == 0:
            return "관련 메뉴 정보를 찾을 수 없습니다."
        
        # 검색 결과를 Document 형식으로 포맷팅
        formatted_docs = "\n\n---\n\n".join(
            [
                f'<Document source="{doc.metadata.get("source", "unknown")}"/>\n{doc.page_content}\n</Document>'
                for doc in docs
            ]
        )
        
        return formatted_docs
    
    except Exception as e:
        return f"메뉴 검색 중 오류가 발생했습니다: {str(e)}"

# 도구 리스트 생성
tools = [search_cafe_menu]

llm = ChatOpenAI(model="gpt-4o-mini")

llm_with_tools = llm.bind_tools(tools)

In [14]:
class AgentState(MessagesState):
    """카페 Agent의 상태를 관리하는 클래스"""
    pass

In [15]:
def create_agent_node():
    """카페 메뉴 상담 Agent 노드를 생성합니다."""
    
    # 시스템 프롬프트 정의
    system_prompt = """당신은 전문적인 카페 메뉴 상담사입니다. 다음 역할을 수행해주세요:

**주요 역할:**
1. 카페 메뉴에 대한 정확하고 친절한 정보 제공
2. 고객의 취향과 예산에 맞는 메뉴 추천
3. 음료와 디저트의 특징, 가격, 제조 방식 설명

**전문 분야:**
- 아메리카노 vs 아이스 아메리카노: 온도와 제조 방식의 차이점
- 라떼 계열 메뉴: 카페라떼, 바닐라 라떼, 녹차 라떼 등의 특징 비교
- 가격대별 추천: ₩4,500~₩7,500 범위의 메뉴 추천
- 디저트 메뉴: 티라미수, 케이크, 쿠키 등의 상세 정보

**응답 가이드라인:**
1. 메뉴 정보가 필요한 경우 search_cafe_menu 도구를 사용하세요
2. 친근하고 전문적인 톤으로 응답하세요
3. 가격 정보와 함께 메뉴의 특징을 설명하세요
4. 고객의 질문에 정확하고 상세하게 답변하세요

메뉴에 대한 구체적인 정보가 필요한 경우, 반드시 search_cafe_menu 도구를 사용하여 정확한 정보를 제공하세요."""

    def agent_node(state: AgentState):
        """Agent 노드 실행 함수"""
        messages = state["messages"]
        
        # 시스템 메시지 추가
        system_message = SystemMessage(content=system_prompt)
        messages_with_system = [system_message] + messages
        
        # LLM 호출
        response = llm_with_tools.invoke(messages_with_system)
        
        return {"messages": [response]}
    
    return agent_node

agent_node = create_agent_node()

In [16]:
def create_cafe_agent():
    """카페 메뉴 상담 Agent 워크플로우를 생성합니다."""
    
    # StateGraph 빌더 생성
    builder = StateGraph(AgentState)
    
    # 노드 추가
    builder.add_node("agent", agent_node)
    builder.add_node("tools", ToolNode(tools))
    
    # 시작점 설정
    builder.add_edge(START, "agent")
    
    # 조건부 엣지 추가 (tools_condition 사용)
    builder.add_conditional_edges(
        "agent",
        tools_condition,  # 자동으로 도구 호출 여부 판단
    )
    
    # 도구 실행 후 다시 agent로 돌아가기
    builder.add_edge("tools", "agent")
    
    # 메모리 설정
    memory = MemorySaver()
    
    # 그래프 컴파일
    graph = builder.compile(checkpointer=memory)
    
    return graph

# 카페 Agent 생성
cafe_agent = create_cafe_agent()

In [17]:
def test_cafe_agent(question: str, thread_id: str = "test-thread"):
    """카페 Agent를 테스트하는 함수"""
    
    print(f"\n{'='*50}")
    print(f"질문: {question}")
    print(f"{'='*50}")
    
    # 설정
    config = {"configurable": {"thread_id": thread_id}}
    
    # Agent 실행
    try:
        for event in cafe_agent.stream(
            {"messages": [HumanMessage(content=question)]},
            config=config,
            stream_mode="values"
        ):
            # 마지막 메시지 출력
            last_message = event["messages"][-1]
            if hasattr(last_message, 'content') and last_message.content:
                if isinstance(last_message, AIMessage):
                    print(f"\n🤖 Agent 응답:")
                    print(last_message.content)
                    print("-" * 50)
    
    except Exception as e:
        print(f"❌ 오류 발생: {str(e)}")

In [18]:
test_cafe_agent("아메리카노와 아이스 아메리카노의 차이점과 가격을 알려주세요.")


test_cafe_agent("라떼 종류에는 어떤 메뉴들이 있고 각각의 특징은 무엇인가요?")


test_cafe_agent("디저트 메뉴 중에서 티라미수에 대해 자세히 설명해주세요.")



질문: 아메리카노와 아이스 아메리카노의 차이점과 가격을 알려주세요.

🤖 Agent 응답:
아메리카노와 아이스 아메리카노는 다음과 같은 차이점이 있습니다.

### 아메리카노
- **가격**: ₩4,500
- **특징**: 에스프레소와 뜨거운 물을 혼합하여 만든 음료로, 커피의 깊은 향과 풍부한 맛을 느낄 수 있습니다. 일반적으로 1:2의 비율로 에스프레소와 물이 섞입니다.
- **제조 방식**: 에스프레소를 추출한 후 뜨거운 물을 추가하여 만듭니다. 보통 커피 본연의 풍미를 강조하며, 뜨거운 음료로 제공됩니다.

### 아이스 아메리카노
- **가격**: ₩4,500
- **특징**: 아메리카노를 만들어서 얼음과 함께 제공하는 음료입니다. 기존의 아메리카노보다 시원하고 산뜻한 느낌을 줍니다.
- **제조 방식**: 역시 에스프레소를 추출한 후 얼음 위에 뜨거운 물을 부어 만듭니다. 이 과정에서 얼음이 녹게 되므로, 물과 커피의 비율이 조절될 수 있습니다.

두 음료 모두 가격이 동일하므로, 개인의 취향에 따라 선택하시면 됩니다. 뜨거운 커피를 원하시면 아메리카노를, 시원한 음료를 원하시면 아이스 아메리카노를 선택하세요!
--------------------------------------------------

질문: 라떼 종류에는 어떤 메뉴들이 있고 각각의 특징은 무엇인가요?

🤖 Agent 응답:
라떼 계열의 대표적인 메뉴들은 다음과 같습니다. 각각의 특징을 살펴보겠습니다.

### 1. 카페라떼
- **가격**: ₩5,500
- **특징**: 에스프레소에 스팀 우유를 추가하여 만든 음료로, 부드럽고 부유한 크리미한 맛이 특징입니다. 에스프레소의 진한 맛과 우유의 부드러움이 조화를 이룹니다.

### 2. 바닐라 라떼
- **가격**: ₩6,000
- **특징**: 기본 카페라떼에 바닐라 시럽이 추가되어 달콤하고 향긋한 맛을 느낄 수 있습니다. 밸런스 좋은 단맛과 커피 맛이 돋보이며, 다양한 디저트와 잘 어울립니다.

### 3. 녹차 라떼
- **가격